In [11]:
import mysql.connector
import hashlib
import datetime
import numpy as np

In [12]:
conn = mysql.connector.connect(host='localhost',database='mysql',user='root',password='Anshul@13')

In [13]:
cursor = conn.cursor()

In [14]:
class Block:
    def __init__(self,data):
        self.data = data
        self.previoushash = None
        self.timestamp = datetime.datetime.now()
        self.hash = self.get_hash()
        
    def get_hash(self):
        header_bin = (str(self.previoushash)+
                     str(self.data)+
                     str(self.timestamp)).encode()
        
        inner_hash = hashlib.sha256(header_bin).hexdigest().encode()
        outer_hash = hashlib.sha256(inner_hash).hexdigest()
        return outer_hash  
    
    def insert(self, data):
        block = Block(data)
        return block.data, self.hash

In [15]:
want_data=input("Do you want to enter data: ")
if want_data == 'y':
    block1 = '0x0'
    choice = 'y'
    sql = "Select Block_Id from block"
    cursor.execute(sql)
    blockss=cursor.fetchall()
    if len(blockss)>1:
        block1=blockss[len(blockss)-1]
        block1=block1[0]

    while choice =='y':
        block = Block(None)
        Input1 = input("Enter data: ")
        Owner = input("Enter Name: ")
        data, Bhash= block.insert(Input1)
        sql= "Insert into block (Block_Id, Previous_Id, Data, Owner, Transaction_Done) values('"+str(Bhash)+"','"+str(block1)+"','"+str(data)+"','"+str(Owner)+"', 'No transaction is Done')"
        cursor.execute(sql)
        conn.commit()
        sql = "Select Block_Id from block"
        cursor.execute(sql)
        blocks=cursor.fetchall()
        block1=blocks[len(blocks)-1]
        block1=block1[0]
        choice = input("Want to add more: ")
    #print (int(len(cursor.fetchall())))


Do you want to enter data: n


In [16]:
choice = input("Do you want to see Ledger: ")
flag = 0
if choice =='y':
    sql = "Select Block_Id from block"
    cursor.execute(sql)
    block_id=cursor.fetchall()
    b_id =input ("Enter your block_id: ")
    block_id = np.array(block_id)
    for i in range(len(block_id)):
        if block_id[i][0]==b_id:
            sql = "Select Data from block"
            cursor.execute(sql)
            data=cursor.fetchall()
            sql = "Select Owner from block"
            cursor.execute(sql)
            owner=cursor.fetchall()
            sql = "Select Transaction_Done from block"
            cursor.execute(sql)
            LT=cursor.fetchall()
            
            print ("Owner - Data - Last Transaction")
            for d in range(len(data)):
                print (owner[d][0]," : ",data[d][0]," : ",LT[d][0])
            flag =1
            break
    if flag ==0:
        print ("Invalid ID")

Do you want to see Ledger: n


In [17]:
trans = input("Do you want to Transaction:")
if trans == 'y':
    sql = "Select Block_Id from block"
    cursor.execute(sql)
    block_id=cursor.fetchall()
    b_id =input ("Enter your block_id: ")
    s_id =input ("Enter id where you transfer: ")
    block_id = np.array(block_id)
    if b_id!=s_id:
        for i in range(len(block_id)):
            if block_id[i][0]==b_id:
                for j in range(len(block_id)):
                    if block_id[j][0]==s_id:
                        sql = "Select Data from block where Block_Id='"+str(b_id)+"'"
                        cursor.execute(sql)
                        data=cursor.fetchall()
                        data = data[0][0]
                        sql = "Select Data from block where Block_Id='"+str(s_id)+"'"
                        cursor.execute(sql)
                        data1=cursor.fetchall()
                        data1 = data1[0][0]
                        sql = "Select Owner from block where Block_Id='"+str(b_id)+"'"
                        cursor.execute(sql)
                        owner=cursor.fetchall()
                        
                        sql = "Select Owner from block where Block_Id='"+str(s_id)+"'"
                        cursor.execute(sql)
                        owner1=cursor.fetchall()
                        
                        trans_amount = int(input("Enter the amount to be transfer: "))
                        if data<trans_amount:
                            print ("Sorry transaction cannot be done due to insufficient amount")
                        else:
                            data = data-trans_amount
                            data1 = data1+trans_amount
                            sql = "Update block set Data='"+str(data)+"' where Block_Id='"+str(b_id)+"'"
                            cursor.execute(sql)
                            conn.commit()
                            sql = "Update block set Data='"+str(data1)+"' where Block_Id='"+str(s_id)+"'"
                            cursor.execute(sql)
                            conn.commit()
                            print ("Transaction is Done from "+str(owner[0][0])+" to "+str(owner1[0][0])+" of Rs."+str(trans_amount))
                            sql = "Update block set Transaction_Done='Debit of Rs."+str(trans_amount)+"' where Block_Id='"+str(b_id)+"'"
                            cursor.execute(sql)
                            conn.commit()
                            sql = "Update block set Transaction_Done='Credit of Rs."+str(trans_amount)+"' where Block_Id='"+str(s_id)+"'"
                            cursor.execute(sql)
                            conn.commit()
                            
                            print ("Transaction is Sucessfully done! :)")
                            
                            
    else:
        print ("Sorry Both ids are same! try again :(")

Do you want to Transaction:y
Enter your block_id: 00ee49bcad84237e83401739ed073a12af0fcc4d58c0aff0bbb63b3c4c676a97
Enter id where you transfer: dd8804b56af1b2767aa4b863433db4b0867b41bc77d21d619a44127fa21f72a8
Enter the amount to be transfer: 10
Transaction is Done from JK to Watson of Rs.10
Transaction is Sucessfully done! :)


In [18]:
'''
sql='TRUNCATE TABLE block'
cursor.execute(sql)
conn.commit()
'''
choice = input("Do you want to close connection: ")
if choice == 'y':
    conn.close()

Do you want to close connection: y
